# 🎵 Audio Restoration Pro

## Princípio Central: "First, Do No Harm"

Este notebook segue as melhores práticas da indústria baseadas em:
- **Bob Katz** - "Mastering Audio: The Art and the Science"
- **iZotope RX** - Padrão da indústria (2x Emmy Awards)
- **IASA** - International Association of Sound and Audiovisual Archives

---

### Filosofia

> "É fácil ir longe demais e fazer mais mal do que bem."

### Workflow

```
1. ANÁLISE     → Entender o problema antes de agir
2. DECISÃO     → Cada etapa é opcional e justificada
3. PROCESSAR   → Mínimo necessário, múltiplos passes suaves
4. VALIDAR     → Comparar A/B, ouvir resíduo, verificar métricas
5. EXPORTAR    → Preservar original, documentar processamento
```

---

## 📦 Etapa 0: Instalação

Instala apenas o necessário. Demucs é **opcional** e instalado separadamente.

In [ ]:
%%capture
# Bibliotecas essenciais
!pip install -q librosa soundfile scipy numpy matplotlib ipywidgets noisereduce

print("✓ Instalação completa!")

## 📂 Etapa 1: Conectar ao Google Drive e Selecionar Arquivo

In [ ]:
from google.colab import drive
import os
import glob
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, HTML, Audio
import numpy as np
import librosa
import soundfile as sf
from datetime import datetime

# Montar Drive
drive.mount('/content/drive')

# Configuração de diretórios
INPUT_DIR = '/content/drive/MyDrive/00-restore'
OUTPUT_DIR = '/content/drive/MyDrive/00-restore/restored'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 Entrada: {INPUT_DIR}")
print(f"📁 Saída: {OUTPUT_DIR}")
print("="*70)

# Buscar arquivos de áudio
audio_files = []
for ext in ['*.mp3', '*.wav', '*.MP3', '*.WAV', '*.m4a', '*.flac', '*.aiff']:
    audio_files.extend(glob.glob(os.path.join(INPUT_DIR, ext)))
audio_files = sorted(set(audio_files))

if not audio_files:
    print(f"⚠️ Nenhum arquivo encontrado em {INPUT_DIR}")
else:
    print(f"✓ {len(audio_files)} arquivo(s) encontrado(s)\n")
    
    # Criar dropdown
    options = [(f"{Path(f).name} ({os.path.getsize(f)/1024/1024:.1f} MB)", f) for f in audio_files]
    
    file_selector = widgets.Dropdown(
        options=options,
        description='Arquivo:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='90%')
    )
    
    display(HTML("<h3>🎵 Selecione o arquivo para restaurar:</h3>"))
    display(file_selector)
    
    # Variável global
    def get_selected_file():
        return file_selector.value

---

## 🔬 Etapa 2: Análise Profunda

**Princípio iZotope**: "Sempre comece analisando o áudio antes de qualquer processamento."

Esta análise irá:
1. Medir SNR (Signal-to-Noise Ratio)
2. Detectar clipping
3. Analisar espectro de frequências
4. Medir loudness (LUFS)
5. Avaliar dynamic range
6. **Recomendar** processamentos (você decide se aplica)

In [ ]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

class AudioAnalyzer:
    """Analisador de áudio seguindo princípios iZotope/Bob Katz"""
    
    def __init__(self, file_path, sr=44100):
        self.file_path = file_path
        self.filename = Path(file_path).name
        self.sr = sr
        
        # Carregar áudio
        print(f"🔍 Carregando: {self.filename}")
        self.y, self.sr = librosa.load(file_path, sr=sr, mono=False)
        
        # Converter para mono para análise
        if self.y.ndim > 1:
            self.y_mono = librosa.to_mono(self.y)
            self.is_stereo = True
        else:
            self.y_mono = self.y
            self.is_stereo = False
        
        self.duration = len(self.y_mono) / self.sr
        self.analysis = {}
        self.recommendations = []
        
    def analyze_all(self):
        """Executa todas as análises"""
        print("="*70)
        print("📊 ANÁLISE COMPLETA DO ÁUDIO")
        print("="*70)
        
        self._analyze_basic()
        self._analyze_clipping()
        self._analyze_noise()
        self._analyze_frequency()
        self._analyze_loudness()
        self._analyze_dynamics()
        self._generate_recommendations()
        self._plot_analysis()
        
        return self.analysis, self.recommendations
    
    def _analyze_basic(self):
        """Informações básicas"""
        print(f"\n📋 INFORMAÇÕES BÁSICAS")
        print(f"   Arquivo: {self.filename}")
        print(f"   Duração: {self.duration:.1f}s ({self.duration/60:.1f} min)")
        print(f"   Sample Rate: {self.sr} Hz")
        print(f"   Canais: {'Estéreo' if self.is_stereo else 'Mono'}")
        
        self.analysis['duration'] = self.duration
        self.analysis['sample_rate'] = self.sr
        self.analysis['is_stereo'] = self.is_stereo
    
    def _analyze_clipping(self):
        """Detectar clipping - PRIMEIRO passo segundo iZotope"""
        print(f"\n🔴 ANÁLISE DE CLIPPING")
        
        # Detectar samples no limite
        threshold = 0.99
        clipped_samples = np.sum(np.abs(self.y_mono) >= threshold)
        total_samples = len(self.y_mono)
        clip_percentage = (clipped_samples / total_samples) * 100
        
        # Detectar sequências de clipping (mais grave)
        clipped_mask = np.abs(self.y_mono) >= threshold
        clip_runs = np.diff(np.where(np.concatenate(([clipped_mask[0]], 
                                                      clipped_mask[:-1] != clipped_mask[1:], 
                                                      [True])))[0])[::2]
        long_clips = np.sum(clip_runs > 3) if len(clip_runs) > 0 else 0
        
        peak = np.max(np.abs(self.y_mono))
        peak_db = 20 * np.log10(peak + 1e-10)
        
        self.analysis['clipping'] = {
            'percentage': clip_percentage,
            'clipped_samples': clipped_samples,
            'long_clips': long_clips,
            'peak': peak,
            'peak_db': peak_db
        }
        
        if clip_percentage > 1:
            print(f"   ⚠️  CLIPPING SEVERO: {clip_percentage:.2f}% dos samples")
            print(f"   ⚠️  {long_clips} sequências longas de clipping")
        elif clip_percentage > 0.1:
            print(f"   ⚠️  Clipping moderado: {clip_percentage:.3f}%")
        else:
            print(f"   ✓ Sem clipping significativo")
        
        print(f"   Peak: {peak_db:.1f} dBFS")
    
    def _analyze_noise(self):
        """Estimar nível de ruído e SNR"""
        print(f"\n📉 ANÁLISE DE RUÍDO")
        
        # Estimar noise floor usando percentis baixos
        noise_floor = np.percentile(np.abs(self.y_mono), 5)
        signal_level = np.percentile(np.abs(self.y_mono), 95)
        
        # SNR estimation
        snr = 20 * np.log10((signal_level + 1e-10) / (noise_floor + 1e-10))
        
        # RMS do ruído (primeiros 500ms se silenciosos, senão percentil baixo)
        first_samples = self.y_mono[:int(0.5 * self.sr)]
        if np.max(np.abs(first_samples)) < 0.1:
            noise_rms = np.sqrt(np.mean(first_samples**2))
        else:
            # Usar frames mais silenciosos
            frame_size = 2048
            frames = librosa.util.frame(self.y_mono, frame_length=frame_size, hop_length=frame_size//2)
            frame_rms = np.sqrt(np.mean(frames**2, axis=0))
            noise_rms = np.percentile(frame_rms, 10)
        
        noise_db = 20 * np.log10(noise_rms + 1e-10)
        
        self.analysis['noise'] = {
            'snr': snr,
            'noise_floor': noise_floor,
            'noise_rms': noise_rms,
            'noise_db': noise_db
        }
        
        print(f"   SNR estimado: {snr:.1f} dB")
        print(f"   Noise floor: {noise_db:.1f} dBFS")
        
        if snr < 15:
            print(f"   ⚠️  RUÍDO ALTO - considere regravar se possível")
        elif snr < 25:
            print(f"   ⚠️  Ruído moderado - redução suave recomendada")
        elif snr < 40:
            print(f"   💡 Ruído leve - redução opcional")
        else:
            print(f"   ✓ Áudio limpo")
    
    def _analyze_frequency(self):
        """Analisar espectro e detectar cortes de frequência"""
        print(f"\n🎼 ANÁLISE ESPECTRAL")
        
        # Calcular espectro
        S = np.abs(librosa.stft(self.y_mono))
        freqs = librosa.fft_frequencies(sr=self.sr)
        
        # Energia por banda
        avg_spectrum = np.mean(S, axis=1)
        
        # Bandas de frequência
        bands = {
            'sub_bass': (20, 60),
            'bass': (60, 250),
            'low_mid': (250, 500),
            'mid': (500, 2000),
            'high_mid': (2000, 4000),
            'presence': (4000, 8000),
            'brilliance': (8000, 16000),
            'air': (16000, 22000)
        }
        
        band_energy = {}
        for name, (low, high) in bands.items():
            mask = (freqs >= low) & (freqs < high)
            if np.any(mask):
                band_energy[name] = np.mean(avg_spectrum[mask])
            else:
                band_energy[name] = 0
        
        total = sum(band_energy.values())
        band_percent = {k: (v/total)*100 for k, v in band_energy.items()}
        
        # Detectar corte de frequências altas
        high_freq_threshold = 8000
        high_mask = freqs >= high_freq_threshold
        high_energy = avg_spectrum[high_mask]
        
        # Encontrar onde energia cai significativamente
        if len(high_energy) > 0:
            normalized = high_energy / (np.max(avg_spectrum) + 1e-10)
            cutoff_idx = np.where(normalized < 0.001)[0]
            if len(cutoff_idx) > 0:
                cutoff_freq = freqs[high_mask][cutoff_idx[0]]
            else:
                cutoff_freq = self.sr / 2
        else:
            cutoff_freq = self.sr / 2
        
        self.analysis['frequency'] = {
            'band_energy': band_energy,
            'band_percent': band_percent,
            'high_freq_cutoff': cutoff_freq,
            'has_high_freq_loss': cutoff_freq < 15000
        }
        
        print(f"   Distribuição de energia:")
        print(f"   Bass (60-250Hz):      {band_percent.get('bass', 0):.1f}%")
        print(f"   Mids (500-2kHz):      {band_percent.get('mid', 0):.1f}%")
        print(f"   Presence (4-8kHz):    {band_percent.get('presence', 0):.1f}%")
        print(f"   Brilliance (8-16kHz): {band_percent.get('brilliance', 0):.1f}%")
        
        if cutoff_freq < 15000:
            print(f"\n   ⚠️  Frequências altas cortadas em ~{cutoff_freq:.0f} Hz")
            if cutoff_freq < 10000:
                print(f"   ⚠️  Perda significativa - pode ser MP3 de baixa qualidade")
        else:
            print(f"\n   ✓ Espectro completo até {cutoff_freq:.0f} Hz")
    
    def _analyze_loudness(self):
        """Análise de loudness (aproximação LUFS)"""
        print(f"\n🔊 ANÁLISE DE LOUDNESS")
        
        # RMS (aproximação de loudness)
        rms = np.sqrt(np.mean(self.y_mono**2))
        
        # Aproximação LUFS (simplificada - para LUFS real usar pyloudnorm)
        # LUFS ≈ 20*log10(RMS) + 0.691 (aproximação)
        lufs_approx = 20 * np.log10(rms + 1e-10)
        
        self.analysis['loudness'] = {
            'rms': rms,
            'lufs_approx': lufs_approx
        }
        
        print(f"   LUFS (aproximado): {lufs_approx:.1f} dB")
        print(f"   RMS: {rms:.4f}")
        
        # Comparar com targets de streaming
        print(f"\n   Comparação com targets de streaming:")
        targets = {'Spotify/YouTube': -14, 'Apple Music': -16, 'Broadcast': -24}
        for platform, target in targets.items():
            diff = lufs_approx - target
            if abs(diff) < 2:
                print(f"   ✓ {platform} ({target} LUFS): OK")
            elif diff > 0:
                print(f"   ⚠️  {platform} ({target} LUFS): {diff:+.1f} dB mais alto")
            else:
                print(f"   💡 {platform} ({target} LUFS): {diff:+.1f} dB mais baixo")
    
    def _analyze_dynamics(self):
        """Análise de dynamic range"""
        print(f"\n📈 ANÁLISE DE DINÂMICA")
        
        # Peak vs RMS
        peak = np.max(np.abs(self.y_mono))
        rms = np.sqrt(np.mean(self.y_mono**2))
        crest_factor = 20 * np.log10(peak / (rms + 1e-10))
        
        # Dynamic range estimation (diferença entre partes mais altas e mais baixas)
        frame_size = 2048
        frames = librosa.util.frame(self.y_mono, frame_length=frame_size, hop_length=frame_size//2)
        frame_rms = np.sqrt(np.mean(frames**2, axis=0))
        
        # Ignorar silêncio
        non_silent = frame_rms > np.percentile(frame_rms, 10)
        if np.any(non_silent):
            loud_rms = np.percentile(frame_rms[non_silent], 95)
            quiet_rms = np.percentile(frame_rms[non_silent], 20)
            dynamic_range = 20 * np.log10((loud_rms + 1e-10) / (quiet_rms + 1e-10))
        else:
            dynamic_range = 0
        
        self.analysis['dynamics'] = {
            'crest_factor': crest_factor,
            'dynamic_range': dynamic_range
        }
        
        print(f"   Crest Factor: {crest_factor:.1f} dB")
        print(f"   Dynamic Range: {dynamic_range:.1f} dB")
        
        if crest_factor < 6:
            print(f"   ⚠️  Muito comprimido (loudness war)")
        elif crest_factor < 10:
            print(f"   💡 Compressão moderada")
        else:
            print(f"   ✓ Boa dinâmica preservada")
    
    def _generate_recommendations(self):
        """Gera recomendações baseadas na análise"""
        print(f"\n" + "="*70)
        print("📋 RECOMENDAÇÕES")
        print("="*70)
        
        self.recommendations = []
        
        # 1. Clipping
        clip = self.analysis['clipping']
        if clip['percentage'] > 0.5:
            self.recommendations.append({
                'priority': 'ALTA',
                'step': 'de-clip',
                'reason': f"Clipping detectado ({clip['percentage']:.2f}%)",
                'action': 'Aplicar de-clipping ANTES de qualquer outro processamento'
            })
        
        # 2. Ruído
        noise = self.analysis['noise']
        if noise['snr'] < 20:
            self.recommendations.append({
                'priority': 'ALTA',
                'step': 'noise-reduction',
                'reason': f"SNR baixo ({noise['snr']:.1f} dB)",
                'action': 'Redução de ruído moderada (0.4-0.6)',
                'warning': 'Se SNR < 10dB, considere regravar'
            })
        elif noise['snr'] < 30:
            self.recommendations.append({
                'priority': 'MÉDIA',
                'step': 'noise-reduction',
                'reason': f"Ruído moderado (SNR: {noise['snr']:.1f} dB)",
                'action': 'Redução de ruído suave (0.2-0.4)'
            })
        
        # 3. Frequências
        freq = self.analysis['frequency']
        if freq['has_high_freq_loss'] and freq['high_freq_cutoff'] < 12000:
            self.recommendations.append({
                'priority': 'BAIXA',
                'step': 'frequency-restoration',
                'reason': f"Frequências cortadas em {freq['high_freq_cutoff']:.0f} Hz",
                'action': 'Excitação de harmônicos (sutil)',
                'warning': 'NÃO é possível recriar frequências perdidas - apenas excitar harmônicos existentes'
            })
        
        # 4. Loudness
        loud = self.analysis['loudness']
        if loud['lufs_approx'] < -20:
            self.recommendations.append({
                'priority': 'MÉDIA',
                'step': 'normalize',
                'reason': f"Áudio muito baixo ({loud['lufs_approx']:.1f} LUFS)",
                'action': 'Normalizar para -14 LUFS (Spotify/YouTube)'
            })
        elif loud['lufs_approx'] > -10:
            self.recommendations.append({
                'priority': 'BAIXA',
                'step': 'check-dynamics',
                'reason': f"Áudio muito alto ({loud['lufs_approx']:.1f} LUFS)",
                'action': 'Verificar se dinâmica foi preservada'
            })
        
        # Mostrar recomendações
        if not self.recommendations:
            print("\n✓ Áudio em bom estado! Processamento mínimo recomendado.")
            print("  Considere apenas normalização para target de streaming.")
        else:
            for i, rec in enumerate(self.recommendations, 1):
                priority_icon = {'ALTA': '🔴', 'MÉDIA': '🟡', 'BAIXA': '🟢'}[rec['priority']]
                print(f"\n{priority_icon} [{rec['priority']}] {rec['step'].upper()}")
                print(f"   Motivo: {rec['reason']}")
                print(f"   Ação: {rec['action']}")
                if 'warning' in rec:
                    print(f"   ⚠️  {rec['warning']}")
        
        print(f"\n" + "="*70)
        print("💡 LEMBRE-SE: Cada processamento é OPCIONAL.")
        print("   Se o áudio já está bom, não processe!")
        print("="*70)
    
    def _plot_analysis(self):
        """Visualização do áudio"""
        fig, axes = plt.subplots(3, 1, figsize=(14, 10))
        
        # 1. Waveform
        times = np.arange(len(self.y_mono)) / self.sr
        axes[0].plot(times, self.y_mono, color='steelblue', linewidth=0.5)
        axes[0].axhline(y=0.99, color='red', linestyle='--', alpha=0.5, label='Clip threshold')
        axes[0].axhline(y=-0.99, color='red', linestyle='--', alpha=0.5)
        axes[0].set_title('Waveform', fontsize=12, fontweight='bold')
        axes[0].set_xlabel('Tempo (s)')
        axes[0].set_ylabel('Amplitude')
        axes[0].legend(loc='upper right')
        axes[0].set_xlim(0, self.duration)
        
        # 2. Spectrogram
        D = librosa.amplitude_to_db(np.abs(librosa.stft(self.y_mono)), ref=np.max)
        img = librosa.display.specshow(D, sr=self.sr, x_axis='time', y_axis='log', ax=axes[1], cmap='magma')
        axes[1].set_title('Espectrograma', fontsize=12, fontweight='bold')
        
        # Marcar cutoff de frequência se houver
        if self.analysis['frequency']['has_high_freq_loss']:
            cutoff = self.analysis['frequency']['high_freq_cutoff']
            axes[1].axhline(y=cutoff, color='cyan', linestyle='--', alpha=0.7, label=f'Cutoff ~{cutoff:.0f}Hz')
            axes[1].legend(loc='upper right')
        
        fig.colorbar(img, ax=axes[1], format='%+2.0f dB')
        
        # 3. Espectro médio
        S = np.abs(librosa.stft(self.y_mono))
        freqs = librosa.fft_frequencies(sr=self.sr)
        avg_spectrum = np.mean(S, axis=1)
        avg_spectrum_db = 20 * np.log10(avg_spectrum + 1e-10)
        
        axes[2].semilogx(freqs[1:], avg_spectrum_db[1:], color='steelblue', linewidth=1)
        axes[2].set_title('Espectro Médio', fontsize=12, fontweight='bold')
        axes[2].set_xlabel('Frequência (Hz)')
        axes[2].set_ylabel('Magnitude (dB)')
        axes[2].set_xlim(20, self.sr/2)
        axes[2].grid(True, alpha=0.3)
        
        # Marcar bandas importantes
        for freq, label in [(250, 'Bass'), (2000, 'Mid'), (8000, 'Presence')]:
            axes[2].axvline(x=freq, color='gray', linestyle=':', alpha=0.5)
            axes[2].text(freq, axes[2].get_ylim()[1]-5, label, fontsize=8, ha='center')
        
        plt.tight_layout()
        plt.savefig('/content/analysis.png', dpi=150, bbox_inches='tight')
        plt.show()
        print("\n📊 Visualização salva em /content/analysis.png")

# Executar análise
if 'file_selector' in dir():
    analyzer = AudioAnalyzer(get_selected_file())
    analysis, recommendations = analyzer.analyze_all()
    
    # Guardar para uso posterior
    AUDIO_ANALYSIS = analysis
    AUDIO_RECOMMENDATIONS = recommendations
    ORIGINAL_AUDIO = analyzer.y_mono
    SAMPLE_RATE = analyzer.sr
    
    print("\n🎧 Ouça o áudio original:")
    display(Audio(analyzer.y_mono, rate=analyzer.sr))
else:
    print("⚠️ Execute a Etapa 1 primeiro para selecionar um arquivo.")

---

## 🔧 Etapa 3: Processamento Modular

Cada módulo é **independente** e **opcional**. 

**Ordem recomendada (iZotope)**:
1. De-clip (se necessário)
2. Redução de ruído (se necessário)
3. EQ corretivo (se necessário)
4. Normalização (quase sempre)

### ⚠️ Regras de Ouro:
- **Menos é mais**: Use configurações mínimas
- **Ouça o resíduo**: Se ouvir música, está removendo demais
- **Compare sempre**: A/B entre original e processado

In [ ]:
class AudioProcessor:
    """Processador de áudio modular seguindo princípios 'First, Do No Harm'"""
    
    def __init__(self, audio, sr):
        self.original = audio.copy()
        self.current = audio.copy()
        self.sr = sr
        self.history = [('original', audio.copy())]
        self.residues = {}  # Para ouvir o que foi removido
    
    def get_current(self):
        return self.current.copy()
    
    def undo(self):
        """Desfaz última operação"""
        if len(self.history) > 1:
            self.history.pop()
            self.current = self.history[-1][1].copy()
            print(f"↩️ Desfeito. Estado atual: {self.history[-1][0]}")
        else:
            print("Nada para desfazer.")
    
    def reduce_noise(self, strength=0.3, stationary=True):
        """
        Redução de ruído conservadora.
        
        Args:
            strength: 0.1 (sutil) a 1.0 (agressivo). Recomendado: 0.2-0.4
            stationary: True para ruído constante (HVAC, hiss)
        """
        import noisereduce as nr
        
        print(f"\n🔇 Aplicando redução de ruído (strength={strength})")
        print(f"   Modo: {'Estacionário' if stationary else 'Não-estacionário'}")
        
        # Aplicar redução
        reduced = nr.reduce_noise(
            y=self.current,
            sr=self.sr,
            prop_decrease=strength,
            stationary=stationary,
            n_fft=2048,
            hop_length=512
        )
        
        # Calcular resíduo (o que foi removido)
        residue = self.current - reduced
        self.residues['noise'] = residue
        
        # Métricas
        residue_energy = np.sqrt(np.mean(residue**2))
        original_energy = np.sqrt(np.mean(self.current**2))
        removed_percent = (residue_energy / original_energy) * 100
        
        print(f"   ✓ Removido: {removed_percent:.1f}% da energia")
        
        if removed_percent > 20:
            print(f"   ⚠️  ATENÇÃO: Muita energia removida! Considere reduzir strength.")
        
        # Atualizar
        self.current = reduced
        self.history.append((f'noise_reduction_{strength}', reduced.copy()))
        
        print(f"   💡 Use player.listen_residue('noise') para ouvir o que foi removido")
    
    def normalize(self, target_db=-14.0, true_peak=-1.0):
        """
        Normalização para target LUFS com proteção de true peak.
        
        Args:
            target_db: Target em dB (aproximação LUFS). -14 = Spotify/YouTube
            true_peak: Limite máximo. -1.0 é padrão da indústria
        """
        print(f"\n🔊 Normalizando para {target_db} dB (true peak: {true_peak} dB)")
        
        # Calcular loudness atual
        current_rms = np.sqrt(np.mean(self.current**2))
        current_db = 20 * np.log10(current_rms + 1e-10)
        
        # Calcular ganho necessário
        gain_db = target_db - current_db
        gain_linear = 10 ** (gain_db / 20)
        
        # Aplicar ganho
        normalized = self.current * gain_linear
        
        # Verificar true peak
        peak = np.max(np.abs(normalized))
        peak_db = 20 * np.log10(peak + 1e-10)
        
        if peak_db > true_peak:
            # Limitar para respeitar true peak
            reduction_db = peak_db - true_peak
            reduction_linear = 10 ** (-reduction_db / 20)
            normalized = normalized * reduction_linear
            print(f"   ⚠️  Peak excedia limite. Reduzido em {reduction_db:.1f} dB")
        
        # Soft clipping para segurança
        normalized = np.tanh(normalized * 0.95) / 0.95
        
        # Métricas finais
        final_rms = np.sqrt(np.mean(normalized**2))
        final_db = 20 * np.log10(final_rms + 1e-10)
        final_peak = 20 * np.log10(np.max(np.abs(normalized)) + 1e-10)
        
        print(f"   Antes: {current_db:.1f} dB")
        print(f"   Depois: {final_db:.1f} dB")
        print(f"   Peak: {final_peak:.1f} dBFS")
        print(f"   ✓ Ganho aplicado: {gain_db:+.1f} dB")
        
        self.current = normalized
        self.history.append((f'normalize_{target_db}', normalized.copy()))
    
    def apply_eq(self, bass=0, mid=0, presence=0, treble=0):
        """
        EQ simples e suave.
        
        Args:
            bass: -6 a +6 dB (centrado em 100Hz)
            mid: -6 a +6 dB (centrado em 1kHz)
            presence: -6 a +6 dB (centrado em 4kHz)
            treble: -6 a +6 dB (centrado em 10kHz)
        """
        from scipy import signal
        
        print(f"\n🎛️ Aplicando EQ: bass={bass:+.1f}, mid={mid:+.1f}, presence={presence:+.1f}, treble={treble:+.1f}")
        
        result = self.current.copy()
        
        # Bandas de EQ
        bands = [
            ('bass', 100, bass),
            ('mid', 1000, mid),
            ('presence', 4000, presence),
            ('treble', 10000, treble)
        ]
        
        for name, freq, gain_db in bands:
            if abs(gain_db) > 0.1:  # Só aplica se ganho significativo
                # Peak filter
                Q = 1.0  # Largura de banda moderada
                w0 = freq / (self.sr / 2)
                
                if w0 < 1.0:  # Frequência válida
                    A = 10 ** (gain_db / 40)
                    alpha = np.sin(np.pi * w0) / (2 * Q)
                    
                    b0 = 1 + alpha * A
                    b1 = -2 * np.cos(np.pi * w0)
                    b2 = 1 - alpha * A
                    a0 = 1 + alpha / A
                    a1 = -2 * np.cos(np.pi * w0)
                    a2 = 1 - alpha / A
                    
                    b = [b0/a0, b1/a0, b2/a0]
                    a = [1, a1/a0, a2/a0]
                    
                    result = signal.filtfilt(b, a, result)
        
        self.current = result
        self.history.append((f'eq', result.copy()))
        print(f"   ✓ EQ aplicado")
    
    def listen_residue(self, step_name):
        """Ouvir o que foi removido em determinada etapa"""
        if step_name in self.residues:
            print(f"🔍 Resíduo de '{step_name}' (o que foi REMOVIDO):")
            print("   Se ouvir música clara aqui, a configuração estava muito agressiva!")
            return Audio(self.residues[step_name], rate=self.sr)
        else:
            print(f"Resíduo '{step_name}' não encontrado.")
            print(f"Disponíveis: {list(self.residues.keys())}")
    
    def compare(self):
        """Comparação A/B entre original e processado"""
        print("\n" + "="*70)
        print("🎧 COMPARAÇÃO A/B")
        print("="*70)
        
        print("\n🔴 ORIGINAL:")
        display(Audio(self.original, rate=self.sr))
        
        print("\n🟢 PROCESSADO:")
        display(Audio(self.current, rate=self.sr))
        
        # Métricas comparativas
        orig_rms = np.sqrt(np.mean(self.original**2))
        proc_rms = np.sqrt(np.mean(self.current**2))
        
        print(f"\n📊 Métricas:")
        print(f"   Original - RMS: {20*np.log10(orig_rms+1e-10):.1f} dB, Peak: {20*np.log10(np.max(np.abs(self.original))+1e-10):.1f} dBFS")
        print(f"   Processado - RMS: {20*np.log10(proc_rms+1e-10):.1f} dB, Peak: {20*np.log10(np.max(np.abs(self.current))+1e-10):.1f} dBFS")
        
        print(f"\n💡 Ouça com atenção:")
        print(f"   - O processado soa MELHOR ou apenas DIFERENTE?")
        print(f"   - Há artefatos (chiados, distorção)?")
        print(f"   - A naturalidade foi preservada?")

# Criar processador
if 'ORIGINAL_AUDIO' in dir():
    processor = AudioProcessor(ORIGINAL_AUDIO, SAMPLE_RATE)
    print("✓ Processador criado!")
    print("\nComandos disponíveis:")
    print("  processor.reduce_noise(strength=0.3)  # Redução de ruído")
    print("  processor.normalize(target_db=-14)    # Normalização")
    print("  processor.apply_eq(bass=0, mid=0, presence=1, treble=0)  # EQ")
    print("  processor.compare()                   # Comparar A/B")
    print("  processor.listen_residue('noise')     # Ouvir o que foi removido")
    print("  processor.undo()                      # Desfazer última operação")
else:
    print("⚠️ Execute a análise primeiro (Etapa 2).")

### 3.1 Redução de Ruído (se recomendado)

**Regra**: Use o valor MÍNIMO que resolve o problema.

In [ ]:
# Ajuste o valor de strength conforme necessário
# 0.1-0.2 = sutil (recomendado para começar)
# 0.3-0.4 = moderado
# 0.5+ = agressivo (cuidado com artefatos!)

processor.reduce_noise(strength=0.25)

In [ ]:
# IMPORTANTE: Ouça o resíduo!
# Se ouvir música clara aqui, está removendo demais!
display(processor.listen_residue('noise'))

### 3.2 EQ Corretivo (opcional)

In [ ]:
# EQ suave - valores em dB (-6 a +6)
# Deixe em 0 as bandas que não precisam de ajuste

processor.apply_eq(
    bass=0,       # 100Hz
    mid=0,        # 1kHz  
    presence=0.5, # 4kHz - adiciona clareza
    treble=0      # 10kHz
)

### 3.3 Normalização

In [ ]:
# Targets comuns:
# -14 dB = Spotify, YouTube, Tidal
# -16 dB = Apple Music
# -24 dB = Broadcast (TV/Rádio)

processor.normalize(target_db=-14.0, true_peak=-1.0)

---

## 🎧 Etapa 4: Comparação Final A/B

In [ ]:
processor.compare()

### Desfazer se necessário

In [ ]:
# Se não ficou bom, desfaça:
# processor.undo()

---

## 💾 Etapa 5: Exportar

Salva o arquivo processado preservando o original.

In [ ]:
# Gerar nome do arquivo de saída
input_name = Path(get_selected_file()).stem
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"{input_name}_restored_{timestamp}.wav"
output_path = os.path.join(OUTPUT_DIR, output_filename)

# Salvar
sf.write(output_path, processor.current, processor.sr, subtype='PCM_24')

print("="*70)
print("✓ ARQUIVO SALVO!")
print("="*70)
print(f"\n📁 Localização: {output_path}")
print(f"📊 Formato: WAV 24-bit")
print(f"🎵 Sample Rate: {processor.sr} Hz")

# Tamanho do arquivo
size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"💾 Tamanho: {size_mb:.1f} MB")

print(f"\n✓ Original preservado em: {get_selected_file()}")

# Log do processamento
print(f"\n📋 Histórico de processamento:")
for i, (step, _) in enumerate(processor.history):
    print(f"   {i}. {step}")

---

## 🎸 Etapa Extra: Separação de Stems (Demucs)

### ⚠️ ATENÇÃO - Leia antes de usar:

**Demucs NÃO deve ser usado para restauração geral!**

Use APENAS se você precisa:
- Isolar vocais para remix
- Extrair bateria ou baixo separadamente
- Remover um instrumento específico

**Problemas conhecidos:**
- Introduz artefatos de separação
- Stem "other" frequentemente tem problemas
- Pode alterar a tonalidade entre segmentos
- O repositório original **não é mais mantido**

Se seu objetivo é apenas **melhorar a qualidade geral**, NÃO use Demucs.

In [ ]:
# Instalar Demucs (apenas se necessário)
# !pip install -q demucs torchaudio

print("⚠️ Demucs não instalado por padrão.")
print("Para instalar, descomente a linha acima e execute.")
print("\nLembre-se: Use apenas se REALMENTE precisar de stems separados!")

---

## 📚 Referências

Este notebook foi construído seguindo:

1. **Bob Katz** - "Mastering Audio: The Art and the Science"
   - K-System para medição de loudness
   - Foco em RMS ao invés de picos

2. **iZotope RX** - Order of Operations
   - Análise primeiro, ação depois
   - Clipping → Ruído steady-state → Ruído complexo → EQ → Normalização

3. **IASA** - International Association of Sound and Audiovisual Archives
   - "Evitar ou minimizar perda de dados"
   - "Transferências sem alterações subjetivas"

4. **Padrões de Streaming (2025)**
   - Spotify/YouTube: -14 LUFS, True Peak -1 dBTP
   - Apple Music: -16 LUFS

---

**Princípio Final**: "Se não está quebrado, não conserte."